In [ ]:

LFW has limitations:
    less woman
    not much side pose and all! 
    trained on aligned faces! see.
    not much of lighting variation
    
so will need to fine-tune on ours.
    
FaceNet: output hi embeddings instead of intermiedate layer! 128 dims. triplet loss
    input changed to 96*96*3 
    

In [80]:
%matplotlib inline

#code to flip upside down raw data

import os
import cv2
import matplotlib.pyplot as plt

reverse_path = r"C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\reverse"

write_folder = r"C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\200108_00" #path should exist!


for img in os.listdir(reverse_path):
    img_path = os.path.join(reverse_path,img)
    image = cv2.imread(img_path)[...,::-1]
    image = cv2.flip(image,0)
    #plt.imshow(image)
    plt.imsave(os.path.join(write_folder,img),image)
    del(image)
    #break

    


In [27]:

%matplotlib inline

import cv2
from mtcnn import MTCNN
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

detector = MTCNN()

read_path = r"C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\top_view_corrected"
#read_path = r"C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\test"

write_path = r"C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\top_view_corrected_face_detected" #path shud exist!
#write_path = read_path+"_face_detected"


for img in os.listdir(read_path):
    img_path = os.path.join(read_path,img)
    #image = cv2.imread(img_path)[...,::-1] #this doesnt seem to work!
    image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

    result = detector.detect_faces(image)
    #print(result)
    
    i=0
    for objects in result:
        bounding_box = objects['box']
        #keypoints = objects['keypoints']
        
        crop_img = image[bounding_box[1]:bounding_box[1]+bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]
        
        directory = os.path.join(write_path,img.split(".")[0])
        if not os.path.exists(directory):
            os.makedirs(directory)
            
        new_path = os.path.join(directory,img.split(".")[0]+"_"+str(i)+".jpg")
        plt.imsave(new_path,crop_img)
        i+=1
        
        cv2.rectangle(image,
                  (bounding_box[0], bounding_box[1]),
                  (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                  (0,155,255),2)
        
    #plt.imshow(image)
    plt.imsave(os.path.join(write_path,img),image)
    
    #break



In [1]:
#Only 1 time, ever!


import bz2
import os

from urllib.request import urlopen

def download_landmarks(dst_file):
    url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
    decompressor = bz2.BZ2Decompressor()
    
    with urlopen(url) as src, open(dst_file, 'wb') as dst:
        data = src.read(1024)
        while len(data) > 0:
            dst.write(decompressor.decompress(data))
            data = src.read(1024)

dst_dir = 'models'
dst_file = os.path.join(dst_dir, 'landmarks.dat')

if not os.path.exists(dst_file):
    os.makedirs(dst_dir)
    download_landmarks(dst_file)

In [ ]:

#Not needed for inference 

"""
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer

# Input for anchor, positive and negative images
in_a = Input(shape=(96, 96, 3))
in_p = Input(shape=(96, 96, 3))
in_n = Input(shape=(96, 96, 3))

# Output for anchor, positive and negative embedding vectors
# The nn4_small model instance is shared (Siamese network)
emb_a = nn4_small2(in_a)
emb_p = nn4_small2(in_p)
emb_n = nn4_small2(in_n)

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([emb_a, emb_p, emb_n])
#alpha is the hyper-parameter

# Model that can be trained with anchor, positive negative images
nn4_small2_train = Model([in_a, in_p, in_n], triplet_loss_layer)
"""


In [36]:
import numpy as np
import os.path

"""

class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in sorted(os.listdir(path)):
        for f in sorted(os.listdir(os.path.join(path, i))):
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)


labelled_path = r"C:\Users\Lenovo\Desktop\Project\DataX-2\labelled Data"
classes = load_metadata(labelled_path)


import matplotlib.pyplot as plt
import matplotlib.patches as patches

import cv2
from align import AlignDlib

%matplotlib inline

def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]

# Initialize the OpenFace face alignment utility
alignment = AlignDlib('models/landmarks.dat')

# Load an image of Jacques Chirac
jc_orig = load_image(metadata[1].image_path())

#print(type(jc_orig))

# Detect face and return bounding box
bb = alignment.getLargestFaceBoundingBox(jc_orig)

# Transform image using specified face landmark indices and crop image to 96x96
jc_aligned = alignment.align(96, jc_orig, bb, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

# Show original image
plt.subplot(131)
plt.imshow(jc_orig)

# Show original image with bounding box
plt.subplot(132)
plt.imshow(jc_orig)
plt.gca().add_patch(patches.Rectangle((bb.left(), bb.top()), bb.width(), bb.height(), fill=False, color='red'))

# Show aligned image
plt.subplot(133)
plt.imshow(jc_aligned);


"""


NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\Lenovo\\Desktop\\Project\\DataX-2\\labelled Data\\Akbar.jpg'

In [57]:
import cv2
from align import AlignDlib
from model import create_model


#alignment = AlignDlib('models/landmarks.dat')
def align_image(img,alignment):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

#img is numpy array of any size!
#output is 96*96*3 for FaceNet to use.

#uses  AlignDlib utility from the OpenFace

nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

#MODEL PRETRAINED ON combinatio of:
# 10,000 subjects and 500,000 images, called CASIAWebFace
# 100,000 Face Images of 530 People. male female balanced called FaceScrub

#200 million images of 8 million identities




In [88]:
import cv2

root="test_images\Extracted_faces"
os.listdir(root)


input_data = []


for image in os.listdir(root):
    if img[-3:] == "jpg" or img[-3:] == "png" or img[-3:] == "jpeg":
        cv2.imread("")
    


class_folders=sorted(os.listdir(root))

for clas in sorted(os.listdir("test_images")):
    for im in os.listdir(os.path.join(root,clas)):
        #clas_images=sorted(os.listdir(clas))[i]
        #print(im)
        metadata.append(IdentityMetadata(root, clas, im))
        
        
    #images_list[i].append(clas_images)
    #metadata.append(IdentityMetadata(root, i, f))
    
metadata=np.array(metadata)
#path=Path(os.path.join(os.getcwd(),"test_images",class_folders[0]),"s1.jpg")




In [74]:

from align import AlignDlib
import random


def distance(emb1, emb2):
    return np.sum(np.square(emb1 - emb2))


def matcher(input_embedding,classes,threshold):
    min_distance = 100.0
    target = -1
    for i in range(classes.shape[0]):
        if distance(input_embedding,classes[i]) < min_distance:
            min_distance = distance(input_embedding,classes[i])
            target = i
            
    if min_distance > threshold: #model unable to classify
        return (-1,round(min_distance,2))
        
    return (target,round(min_distance,2))

    

def pipeline(input_path,classes,result_path,class_names,threshold=0.8,random_input=30):
    # Initialize the OpenFace face alignment utility
    alignment = AlignDlib('models/landmarks.dat')
    
    input_dim = 96

    #X_train = np.ones(shape=(tot,input_dim,input_dim),dtype=np.uint8)

    #random_input = 100
    input_items = os.listdir(input_path)
    input_folders = filter(lambda x: x[-3:] !="jpg", input_items) 
    input_folders = list(input_folders)
    
    tot = len(input_folders)
    if tot > random_input:
        #input_folders = random.sample(input_folders,random_input)
        input_folders = input_folders[:random_input]
    
    #embedded = np.zeros((tot, 128))
    
    k = len(input_folders)
    for j,folder in enumerate(input_folders):
        print("Image %d out of %d" %(j,k))
        #anything reset?
        folder_path = os.path.join(input_path,folder)
        print(folder_path)
        for i,file in enumerate(os.listdir(folder_path)):
            
            im = cv2.imread(os.path.join(folder_path,file))
            
            im = im[...,::-1]
            #im_new = cv2.resize(im, (96,96), interpolation=cv2.INTER_LINEAR) #cubic increase all the values!
            
            im_new = align_image(im,alignment) #also changes dimension to std 96*96
            
            if np.any(im_new) == None: #unable to align image
                continue
                
            #print(im.shape,im_new.shape)
            
            #src, dst, dst.size(), 0, 0, interpolation
            #X_train[i] = im_new
            im_new = (im_new / 255.).astype(np.float32)
            embedding = nn4_small2_pretrained.predict(np.expand_dims(im_new, axis=0))[0]

            target,distance = matcher(embedding,classes,threshold)
            
            if target == -1: #model unable to classify
                if not os.path.exists(os.path.join(result_path,"unclassified",folder)):
                    os.makedirs(os.path.join(result_path,"unclassified",folder))
                
                plt.imsave(os.path.join(os.path.join(result_path,"unclassified",folder),file[:-4]+str(distance)+".jpg"),im)
            else:
                if not os.path.exists(os.path.join(result_path,folder)):
                    os.makedirs(os.path.join(result_path,folder))
                    
                plt.imsave(os.path.join(os.path.join(result_path,folder),class_names[target]+str(distance)+".jpg"),im)
            
            #print(target,distance)
    
    #return embedded
    #alignment.align(96, img, alignment.getLargestFaceBoundingBox(img),landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
    

In [78]:

labelled_path = r"C:\Users\Lenovo\Desktop\Project\DataX-2\labelled Data"
input_path = r"C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\top_view_corrected_face_detected" #dont read images from here!
result_path = r"C:\Users\Lenovo\Desktop\Project\DataX-2\Results"

class_names = os.listdir(r"C:\Users\Lenovo\Desktop\Project\DataX-2\labelled Data")
class_names = [item[:-4] for item in class_names]

embedded_test = pipeline(input_path,classes_embedded,result_path,class_names,threshold=0.8,random_input=10)
#"test_images\Extracted_faces"

#.43 wrong snehdeep classified as Manish, .33 vinay as anurag
#on the ones it can classify: 11/15 

#total:194. classified:53,unclassified:7. means couldn't align: 134!



Image 0 out of 10
C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\top_view_corrected_face_detected\000005
(44, 31, 3) (96, 96, 3)
(32, 27, 3) (96, 96, 3)
(50, 43, 3) (96, 96, 3)
(36, 30, 3) (96, 96, 3)
Image 1 out of 10
C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\top_view_corrected_face_detected\000007
(31, 28, 3) (96, 96, 3)
(38, 30, 3) (96, 96, 3)
(68, 53, 3) (96, 96, 3)
(49, 39, 3) (96, 96, 3)
Image 2 out of 10
C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\top_view_corrected_face_detected\000008
(47, 40, 3) (96, 96, 3)
(35, 30, 3) (96, 96, 3)
(32, 29, 3) (96, 96, 3)
(53, 42, 3) (96, 96, 3)
(38, 31, 3) (96, 96, 3)
(51, 40, 3) (96, 96, 3)
(37, 30, 3) (96, 96, 3)
(47, 38, 3) (96, 96, 3)
Image 3 out of 10
C:\Users\Lenovo\Desktop\Project\DataX-2\Raw Data\top_view_corrected_face_detected\000009
(68, 48, 3) (96, 96, 3)
(75, 64, 3) (96, 96, 3)
(32, 29, 3) (96, 96, 3)
(69, 51, 3) (96, 96, 3)
(50, 40, 3) (96, 96, 3)
(39, 31, 3) (96, 96, 3)
Image 4 out of 10
C:\Users\Lenovo\Desktop\Pr

In [79]:
class_names

['Akbar',
 'Aman',
 'Anisha',
 'Anshuman',
 'Anurag',
 'Balbir',
 'Bharadeep',
 'Hemant',
 'Kartik',
 'Manan',
 'Manish',
 'Ojas',
 'Ojasvi',
 'Pradeep',
 'Praveen',
 'Priyanka',
 'Ramya',
 'Ravinder_LI',
 'Sai',
 'Saif',
 'Shrey Arora',
 'Snehdeep',
 'Tushar',
 'Vinay']